# Analysis of Swarm MAG LR data (1Hz)

For more information about the data and other ways to access it, see:

- [Data handbook](https://swarmhandbook.earth.esa.int/catalogue/SW_MAGx_LR_1B)
- [viresclient documentation](https://viresclient.readthedocs.io/en/latest/available_parameters.html)
- [MAGx_LR demo notebook](https://notebooks.vires.services/notebooks/03a1_demo-magx_lr_1b)

In [ ]:
import datetime as dt
import matplotlib.pyplot as plt

from swarmpal.io import create_paldata, PalDataItem
from swarmpal.toolboxes import tfa

## Fetching data

As in the introduction example, we will fetch the MAG LR data.

In [ ]:
data = create_paldata(
    PalDataItem.from_vires(
        collection="SW_OPER_MAGA_LR_1B",
        measurements=["B_NEC"],
        models=["Model=CHAOS"],
        auxiliaries=["QDLat", "MLT"],
        start_time=dt.datetime(2015, 3, 14),
        end_time=dt.datetime(2015, 3, 14, 3, 59, 59),
        pad_times=(dt.timedelta(hours=3), dt.timedelta(hours=3)),
        server_url="https://vires.services/ows",
        options=dict(asynchronous=False, show_progress=False),
    )
)
print(data)

## Processing

This time we will use the `convert_to_mfa` option to rotate the B_NEC vector to the mean-field aligned (MFA) frame. When the MFA frame is used, the `active_component` must be set to one of (0, 1, 2): 0 is the poloidal component, 1 the toroidal and 2 the compressional. Similarly for B_NEC, the numbers correspond to the North (0), East (1) or Center (2) components.

In [ ]:
p1 = tfa.processes.Preprocess()
p1.set_config(
    dataset="SW_OPER_MAGA_LR_1B",
    active_variable="B_MFA",
    active_component=2,
    sampling_rate=1,
    remove_model=True,
    convert_to_mfa=True,
)
p1(data);

Even though `B_MFA` isn't available in the original data, this variable becomes available when we select `convert_to_mfa=True`. For more information on the other options, refer to the documentation:

In [ ]:
help(tfa.processes.Preprocess.set_config)

We can inspect the data directly to get an idea about what has happened using `Preprocess`.

In the figure below, the input `B_NEC` (first row) and `B_NEC_Model` have been taken to produce `B_NEC_res_Model` (second row), and then that has been rotated to the MFA frame (third row). Component "2" is identified from B_MFA and used as the TFA variable (`active_component=2` in the above config).

In [ ]:
fig, axes = plt.subplots(3, 1, sharex=True)
data["SW_OPER_MAGA_LR_1B"]["B_NEC"].plot.line(x="Timestamp", ax=axes[0])
data["SW_OPER_MAGA_LR_1B"]["B_NEC_res_Model"].plot.line(x="Timestamp", ax=axes[1])
data["SW_OPER_MAGA_LR_1B"]["B_MFA"].plot.line(x="Timestamp", ax=axes[2])
axes[1].set_ylim(-200, 200)
axes[2].set_ylim(-200, 200);

Let's prepare the other processes...

In [ ]:
p2 = tfa.processes.Clean()
p2.set_config(
    window_size=300,
    method="iqr",
    multiplier=1,
)
p3 = tfa.processes.Filter()
p3.set_config(
    cutoff_frequency=20 / 1000,
)
p4 = tfa.processes.Wavelet()
p4.set_config(
    min_frequency=20 / 1000,
    max_frequency=100 / 1000,
    dj=0.1,
)

In practice, you might want to prepare and apply each process in turn to make sure things work right. Here however, we will just apply them all together. Make sure you apply them in the right order!

In [ ]:
p2(data)
p3(data)
p4(data);

## Plotting

In [ ]:
tfa.plotting.quicklook(data);